In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import holodeck.detstats as ds
import h5py

In [ ]:
hdf_name = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/ss_lib.hdf5'
detstats_path = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/detstats'

npsrs = 40
sigma = 1e-7
output_dir = detstats_path+'/ds2_psrs40_sigma1e-7'

In [ ]:
vals = ds.detect_lib(hdf_name, output_dir, npsrs, sigma, 
                     nskies=25, thresh=0.5, plot=True, debug=True)